In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Importing Libraries**

In [ ]:
#!pip install flair
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
#from flair.models import TextClassifier
#from flair.data import Sentence
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk
'''
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
'''
from wordcloud import WordCloud
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
import warnings
warnings.filterwarnings("ignore")

****

**Importing Dataset**

In [ ]:
data = pd.read_csv('../input/guitar2ugh/guitar3000.csv', sep = ',')
df = data.copy()
print ("Dataset: ", data.head())

**Cleaning The Dataset**

In [ ]:
'''
stop_words = set(stopwords.words("english"))
df["Review"] = df["Review"].str.replace("\d","")

def cleaner(data):
    # Tokens
    tokens = word_tokenize(str(data).replace("'", "").lower())
    # Remove Puncs
    without_punc = [w for w in tokens if w.isalpha()]
    # Stopwords
    without_sw = [t for t in without_punc if t not in stop_words]
    # Lemmatize
    text_len = [WordNetLemmatizer().lemmatize(t) for t in without_sw]
    # Stem
    text_cleaned = [PorterStemmer().stem(w) for w in text_len]
    return " ".join(text_cleaned)

df["Review"] = df["Review"].apply(cleaner)
df["Review"]
'''
'''
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

df['Review']=df['Review'].apply(lambda x: remove_punctuation(x))
#df.head()

def tokenize(text):
    split=re.split("\W+",text) 
    return split

df['Review']=df['Review'].apply(lambda x: tokenize(x.lower()))
#df.head()
stopword = nltk.corpus.stopwords.words('english')
print(stopword[:11])

def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text
df['Review'] = df['Review'].apply(lambda x: remove_stopwords(x))

df.head()
'''

**Assigning independent and dependent variables**

In [ ]:
X=data['Review'].values
Y=data['Sug/Com']
#Y=data['Vader'].values
#Y=data['TextBlob_Polarity'].values
#Y=data['flair_sentiment'].values
#Y = data['Suggestion/Complaint'].values
print(X.shape)
print(Y.shape)

**Split the dataset/ Bag of Words**

In [ ]:
kf = KFold(n_splits = 5, shuffle = True)
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=42)

**Apply Bag of words Features on Splitted data**

In [ ]:
Vect=CountVectorizer()

**Decision Tree and Training**

In [ ]:
model=DecisionTreeClassifier(max_depth=6, class_weight='balanced')

scores_dectree = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    model = model.fit(Bow_train, y_train)
    predictions = model.predict(Bow_test)
    scores_dectree.append(model.score(Bow_test,y_test))
print('Scores from each Iteration: ', scores_dectree)
print('Average K-Fold Score :' , np.mean(scores_dectree))

**Naive Bayes ML Algorithm**

In [ ]:
sc = StandardScaler(with_mean=False)

# Training the Naive Bayes model on the Training set
classifier = GaussianNB()

scores_naive = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    X_train_sc = sc.fit(Bow_train)
    X_Train = X_train_sc.transform(Bow_train)
    X_Test = X_train_sc.transform(Bow_test)
    X_Train = X_Train.toarray()
    classifier = classifier.fit(X_Train, y_train)
    X_Test = X_Test.toarray()
    predictions = classifier.predict(X_Test)
    scores_naive.append(classifier.score(X_Test,y_test))
print('Scores from each Iteration: ', scores_naive)
print('Average K-Fold Score :' , np.mean(scores_naive))

**SVM Machine Learning Algorithm**

In [ ]:
st_x= StandardScaler(with_mean=False)   
x_train= st_x.fit_transform(Bow_train)    
x_test= st_x.transform(Bow_test)

#Training the SVM model
classifier = SVC(kernel='linear', random_state=0)

scores_svm = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    x_train= st_x.fit_transform(Bow_train)    
    x_test= st_x.transform(Bow_test)
    classifier = classifier.fit(x_train, y_train)
    predictions = classifier.predict(x_test)
    scores_svm.append(classifier.score(x_test,y_test))
print('Scores from each Iteration: ', scores_svm)
print('Average K-Fold Score :' , np.mean(scores_svm))

**XG Boosting**

In [ ]:
from xgboost import XGBClassifier

st_x= StandardScaler(with_mean=False)   

# fit model no training data
model = XGBClassifier()

scores_xgb = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    x_train= st_x.fit_transform(Bow_train)    
    x_test= st_x.transform(Bow_test)
    model = model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    scores_xgb.append(model.score(x_test,y_test))
print('Scores from each Iteration: ', scores_xgb)
print('Average K-Fold Score :' , np.mean(scores_xgb))

**Bag Of Words + XG Boosting**

In [ ]:
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV

st_x= StandardScaler(with_mean=False)   

xgb = XGBClassifier()
model = BaggingClassifier(base_estimator=xgb, n_estimators=31, random_state=314)

scores_bagxgb = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    x_train= st_x.fit_transform(Bow_train)    
    x_test= st_x.transform(Bow_test)
    model = model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    scores_bagxgb.append(model.score(x_test,y_test))
print('Scores from each Iteration: ', scores_bagxgb)
print('Average K-Fold Score :' , np.mean(scores_bagxgb))

**Using Hyper Parameters on SVD Model**

In [ ]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
scores_hypersvd = []    
    
for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    x_train= st_x.fit_transform(Bow_train)    
    x_test= st_x.transform(Bow_test)
    grid = grid.fit(x_train, y_train)
    predictions = grid.predict(x_test)
    scores_hypersvd.append(grid.score(x_test,y_test))
print('Scores from each Iteration: ', scores_hypersvd)
print('Average K-Fold Score :' , np.mean(scores_hypersvd))

**Bagging On SVD**

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV

#import hasy_tools 

# Define model
#svm = LinearSVC(random_state=42)
svm = SVC(kernel='linear', random_state=0) 
model = BaggingClassifier(base_estimator=svm, n_estimators=31, random_state=314)

scores_bagsvd = []

for train_index, test_index in rkf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Bow_train=Vect.fit_transform(x_train)
    Bow_test=Vect.transform(x_test)
    x_train= st_x.fit_transform(Bow_train)    
    x_test= st_x.transform(Bow_test)
    model = model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    scores_bagsvd.append(model.score(x_test,y_test))
print('Scores from each Iteration: ', scores_bagsvd)
print('Average K-Fold Score :' , np.mean(scores_bagsvd))

**Averages of Cross Validation of All Models**

In [ ]:
x = [1, 2, 3, 4]

print('Decision Tree:')
print('Scores from each Iteration: ', scores_dectree)
print('Average K-Fold Score :' , np.mean(scores_dectree))

print('Naive Bayes:')
print('Scores from each Iteration: ', scores_naive)
print('Average K-Fold Score :' , np.mean(scores_naive))

print('SVM:')
print('Scores from each Iteration: ', scores_svm)
print('Average K-Fold Score :' , np.mean(scores_svm))

print('XG Boosting')
print('Scores from each Iteration: ', scores_xgb)
print('Average K-Fold Score :' , np.mean(scores_xgb))

print('XG Boosting with Bagging: ')
print('Scores from each Iteration: ', scores_bagxgb)
print('Average K-Fold Score :' , np.mean(scores_bagxgb))

print("SVD with HyperParameters")
print('Scores from each Iteration: ', scores_hypersvd)
print('Average K-Fold Score :' , np.mean(scores_hypersvd))

print("Bagging with SVD")
print('Scores from each Iteration: ', scores_bagsvd)
print('Average K-Fold Score :' , np.mean(scores_bagsvd))

plt.rcParams["figure.figsize"] = (17,8)

# Plot a simple line chart
plt.plot(x, scores_dectree, label='Decision Tree')
plt.plot(x, scores_naive, label='Naive Bayes')
plt.plot(x, scores_xgb, label='XG Boosting')
plt.plot(x, scores_bagxgb, label='XG Boosting with Bagging')
plt.plot(x, scores_svm, label='SVM')
plt.plot(x, scores_hypersvd, label='SVD with Hyper Parameters')
plt.plot(x, scores_bagsvd, label='SVD with Bagging')
plt.ylabel('Accuracy Score', fontsize = 10)
plt.xlabel('Samples', fontsize = 10)
plt.legend()
plt.show()